# Aprendizem Profunda - DNN
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [139]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization, LeakyReLU, Input
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline
)


### Input dos dados

In [140]:
df = pd.read_csv("data/small_dataset.csv")
df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")


### Função pra gravar em csv

In [141]:
def to_csv(results_df, name):
    os.makedirs("submissao2", exist_ok=True)
    path = f"submissao2/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [142]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [143]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 4053, Unique texts: 4051
Overlap between train and test after deduplication: 0


### Tokenizer

In [144]:
# Tokenize the data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=100)
X_val = pad_sequences(X_val_seq, maxlen=100)
X_test = pad_sequences(X_test_seq, maxlen=100)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [145]:
data_set_professor = pd.read_csv("data/dataset3_inputs.csv", sep=';', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=100)  

### Early Stopping

In [146]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=10,            
    restore_best_weights=True,           
)

### Reduce_lr

In [147]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3,
    min_lr=0.00005
)

## DNN

In [148]:
model_dnn = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(64, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(32, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(1, activation="sigmoid"),
])

# Compile
model_dnn.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

# Train
history = model_dnn.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    shuffle=True
)

Epoch 1/200


c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


81/81 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5050 - loss: 1.1054 - val_accuracy: 0.5849 - val_loss: 0.9203 - learning_rate: 1.0000e-04
Epoch 2/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5575 - loss: 1.0278 - val_accuracy: 0.6250 - val_loss: 0.8953 - learning_rate: 1.0000e-04
Epoch 3/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5512 - loss: 1.0320 - val_accuracy: 0.6682 - val_loss: 0.8777 - learning_rate: 1.0000e-04
Epoch 4/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5911 - loss: 0.9683 - val_accuracy: 0.6806 - val_loss: 0.8636 - learning_rate: 1.0000e-04
Epoch 5/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5832 - loss: 0.9837 - val_accuracy: 0.6821 - val_loss: 0.8527 - learning_rate: 1.0000e-04
Epoch 6/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5907 - loss: 0.9669 - val_accuracy: 0.6883 - val_loss: 0.8426 - learning_rate: 1.0000e-04
Epoch 7/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5954 - loss: 0.95

## Simple RNN

# Correr Modelos

**Test Set**

In [149]:
test_ids = df.loc[test_indices, "ID"]  # Get IDs for test split only
predict_and_save(model_dnn, X_test, test_ids, "dnn")
model_dnn.save("models/dnn.keras")  

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


**Avaliação**

In [150]:
predict_and_save(model_dnn, X_professor, data_set_professor["ID"], "dnn_professor")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


## Classification Reports

In [151]:
print(classification_report(y_test, model_dnn.predict(X_test) > 0.5))

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
              precision    recall  f1-score   support

         0.0       0.78      0.64      0.70       420
         1.0       0.68      0.80      0.73       391

    accuracy                           0.72       811
   macro avg       0.73      0.72      0.72       811
weighted avg       0.73      0.72      0.72       811

